In [40]:
from langgraph.graph import StateGraph,START,END
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from typing import TypedDict,Annotated
from pydantic import BaseModel,Field
import operator


In [41]:
load_dotenv()

True

In [42]:
model = ChatOpenAI(model='gpt-4o-mini')

In [69]:
class EvaluationSchema(BaseModel):

    feedback: str = Field(description = "Detailed feedback for essay")
    score: int = Field(description="score essay on a scale of 1 to 10", ge=0, le=10)

In [56]:
structutred_model = model.with_structured_output(EvaluationSchema)

In [58]:
essay = """Artificial Intelligence (AI) plays a transformative role in human development by enhancing efficiency, decision-making, and innovation across various sectors. In healthcare, AI accelerates diagnosis, personalizes treatments, and improves patient outcomes. In education, it enables adaptive learning systems that cater to individual needs, bridging knowledge gaps and promoting lifelong learning. AI also contributes to economic growth by automating repetitive tasks, allowing humans to focus on creativity and strategic thinking. Furthermore, AI supports sustainable development by optimizing energy usage, improving agricultural productivity, and aiding climate research. Its integration in social services helps predict and address societal challenges such as poverty, inequality, and mental health. However, ethical deployment is crucial to ensure inclusivity, privacy, and transparency. As AI continues to evolve, it holds immense potential to empower humanity, fostering a more intelligent, connected, and equitable world. The key lies in responsible innovation and global cooperation to harness AI for the collective good of society"""

In [59]:
prompt  = f'Evaluate the following essay and provide detailed feedback and a score on a scale of 1 to 10:\n\n{essay}'
structured_output = structutred_model.invoke(prompt)
print(structured_output)

feedback="The essay presents a well-rounded discussion on the transformative role of Artificial Intelligence (AI) in various sectors. It clearly outlines how AI enhances efficiency, decision-making, and innovation in areas such as healthcare, education, and economic development. Each sector is addressed succinctly, with relevant examples that illustrate the benefits of AI. Furthermore, the inclusion of potential ethical concerns surrounding AI's deployment adds depth to the argument, showing an awareness of the challenges that come with technological advancement. However, the essay could benefit from a more robust conclusion that summarizes the main points more cohesively instead of simply reiterating the importance of responsible innovation. Additionally, exploring specific examples or case studies could enhance the arguments presented, making them more compelling. Overall, the writing is coherent and well-structured, but it would be stronger with more concrete examples and a more def

In [60]:
class UPSCState(TypedDict):
    essay:str
    language_feedback:str
    analysis_feedback:str
    clarity_feedback:str
    overall_feedback:str
    score: Annotated[list[int], operator.add]
    avg_score:float

In [61]:
def evaluate_language(state: UPSCState) -> UPSCState:
    prompt = prompt  = f'Evaluate the language quality following essay and provide detailed feedback and a score on a scale of 1 to 10:\n\n{state['essay']}'
    structured_output = structutred_model.invoke(prompt)
    return {'language_feedback': structured_output.feedback,'individual_score':{structured_output.score}}

In [62]:
def evaluate_analysis(state: UPSCState) -> UPSCState:
    prompt = prompt  = f'Evaluate the depth of analysis following essay and provide detailed feedback and a score on a scale of 1 to 10:\n\n{state['essay']}'
    structured_output = structutred_model.invoke(prompt)
    return {'analysis_feedback': structured_output.feedback,'individual_score':{structured_output.score}}

In [51]:
def evaluate_thought(state: UPSCState) -> UPSCState:
    prompt = prompt  = f'Evaluate the calarity of thought following essay and provide detailed feedback and a score on a scale of 1 to 10:\n\n{state['essay']}'
    structured_output = structutred_model.invoke(prompt)
    return {'clarity_feedback': structured_output.feedback,'individual_score':{structured_output.score}}

In [66]:
def final_evaluation(state:UPSCState):
    #summary feedback
    prompt = f'Based on the following feedback, provide a summary feedback \n language feedback:{state["language_feedback"]} \n depth of analysis feedback:{state["analysis_feedback"]} \n calarity of thought feedback:{state["clarity_feedback"]}'
    overall_feedback = model.invoke(prompt).content
    #Calculate average score
    if 'score' in state and len(state['score']) > 0:
        avg_score = sum(state['score']) / len(state['score'])
    else:
        avg_score = 0 
    return {'overall_feedback': overall_feedback, 'avg_score': avg_score}

In [67]:
graph = StateGraph(UPSCState)

#Add graph
graph.add_node('evaluate_language',evaluate_language)
graph.add_node('evaluate_analysis',evaluate_analysis)
graph.add_node('evaluate_thought',evaluate_thought)
graph.add_node('final_evaluation',final_evaluation)
#Add edges
graph.add_edge(START,'evaluate_language')
graph.add_edge(START,'evaluate_analysis')
graph.add_edge(START,'evaluate_thought')

graph.add_edge('evaluate_language','final_evaluation')
graph.add_edge('evaluate_analysis','final_evaluation')
graph.add_edge('evaluate_thought','final_evaluation')

graph.add_edge('final_evaluation', END)

#Comile the graph
workflow = graph.compile()








In [70]:
intial_state = {
    'essay':essay,
}

final_workflow = workflow.invoke(intial_state)
print(final_workflow)

{'essay': 'Artificial Intelligence (AI) plays a transformative role in human development by enhancing efficiency, decision-making, and innovation across various sectors. In healthcare, AI accelerates diagnosis, personalizes treatments, and improves patient outcomes. In education, it enables adaptive learning systems that cater to individual needs, bridging knowledge gaps and promoting lifelong learning. AI also contributes to economic growth by automating repetitive tasks, allowing humans to focus on creativity and strategic thinking. Furthermore, AI supports sustainable development by optimizing energy usage, improving agricultural productivity, and aiding climate research. Its integration in social services helps predict and address societal challenges such as poverty, inequality, and mental health. However, ethical deployment is crucial to ensure inclusivity, privacy, and transparency. As AI continues to evolve, it holds immense potential to empower humanity, fostering a more intell